In [8]:
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from datetime import datetime

In [9]:
# Загрузка предварительно обученной модели ResNet50
model = ResNet50(weights='imagenet', include_top=False)

In [10]:
# Функция для загрузки и предобработки изображения
def preprocess_image(img_array):
    img = image.array_to_img(img_array, scale=False)
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

In [11]:
def compare_frames(frame1, frame2, threshold):
    # Преобразование кадров в градации серого
    gray_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Сравнение среднеквадратичной ошибки между изображениями
    mse = np.mean((gray_frame1 - gray_frame2) ** 2)
    
    # Если среднеквадратичная ошибка мала, то кадры считаются схожими
    return mse < threshold, mse  # Возвращаем также значение MSE

In [12]:
original_video_path = 'VideoOriginal.mp4'
desync_video_path = 'Video.mp4'

In [13]:
original_video_capture = cv2.VideoCapture(original_video_path)
desync_video_capture = cv2.VideoCapture(desync_video_path)

In [14]:
original_video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
desync_video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

True

In [15]:
# Открытие файл логов
log_file = open('History.txt', 'w')

In [16]:
# Чтение первого кадра для обоих видеопотоков
original_ret, original_frame = original_video_capture.read()
desync_ret, desync_frame = desync_video_capture.read()

In [17]:
# Установка начального времени
current_time = 0

In [18]:
while original_ret and desync_ret:
    # Сравнение кадров на текущем времени
    threshold = 10
    match, mse = compare_frames(original_frame, desync_frame, threshold)
    
    # Запись результата в файл логов
    log_file.write(f'{datetime.now()} - Кадры {"совпали" if match else "не совпали"}. MSE: {mse} на {current_time} секунде видео\n')
    
    # Переход к следующему времени в видеопотоках
    current_time += 2
    original_video_capture.set(cv2.CAP_PROP_POS_MSEC, current_time * 1000)
    desync_video_capture.set(cv2.CAP_PROP_POS_MSEC, current_time * 1000)
    
    # Считывание следующих кадров
    original_ret, original_frame = original_video_capture.read()
    desync_ret, desync_frame = desync_video_capture.read()

In [19]:
# Закрытие файл логов
log_file.close()

# Освобождение ресурсов
original_video_capture.release()
desync_video_capture.release()

print("Сравнение завершено. Результаты записаны в файл логов.")

Сравнение завершено. Результаты записаны в файл логов.
